# A2A リモートエージェント対応のマルチエージェントシステム

## 環境準備

In [ ]:
%pip install --user \
    google-adk==1.11.0 \
    a2a-sdk==0.3.1

In [18]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

In [1]:
import copy, json, os, uuid
import vertexai
from vertexai import agent_engines

from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.sequential_agent import SequentialAgent

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [2]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## リモートエージェントの定義と Agent Engine へのデプロイ

In [4]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [5]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [6]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

レビュアーによる修正ポイントが指摘された際は、直前に書いた記事を指摘に従って書き直してください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [7]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

In [ ]:
agents = {
    'research_agent1': research_agent1,
    'research_agent2': research_agent2,
    'writer_agent': writer_agent,
    'review_agent': review_agent,
}

deployed_agents = [agent.display_name for agent in agent_engines.list()]

for agent_name in agents.keys():
    if not agent_name in deployed_agents:
        remote_agent = agent_engines.create(
            agent_engine=agents[agent_name],
            display_name=f'{agent_name}',
            requirements=['google-adk==1.11.0'],
        )
    else:
        resource_name = [
            agent.name for agent in agent_engines.list()
            if agent.display_name == agent_name
        ][0]
        remote_agent = agent_engines.update(
            resource_name=resource_name,
            agent_engine=agents[agent_name],
            display_name=f'{agent_name}',
            requirements=['google-adk==1.11.0'],
        )

In [11]:
agent_list = list(agent_engines.list())
for agent_name in agents.keys():
    resource_name = [
        agent.name for agent in agent_list
        if agent.display_name == agent_name
    ][0]
    print(f'{agent_name}: {resource_name}')

research_agent1: 3354750713832931328
research_agent2: 4183413045269102592
writer_agent: 180557401466863616
review_agent: 4784643595523063808


## A2A サーバーのデプロイ

In [13]:
REPO_NAME='cloud-run-source-deploy'
REPO=f'{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}'
!gcloud artifacts repositories create {REPO_NAME} \
     --repository-format docker --location {LOCATION}

Create request issued for: [cloud-run-source-deploy]
Waiting for operation [projects/etsuji-a2a-agents/locations/us-central1/operati
ons/d92efbeb-faa4-4597-9350-ffe7af5eb8d8] to complete...done.                  
Created repository [cloud-run-source-deploy].


In [14]:
%%bash
cat <<EOF >main.py
import os
import vertexai
from vertexai import agent_engines

from a2a.server.agent_execution import AgentExecutor
from a2a.server.tasks import TaskUpdater, InMemoryTaskStore
from a2a.types import (
    Task, TextPart, UnsupportedOperationError,
    AgentCapabilities, AgentCard, AgentSkill,
)
from a2a.utils.errors import ServerError
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler

PROJECT_ID = os.environ['PROJECT_ID']
LOCATION = os.environ['REGION']
AGENT_ID = os.environ['AGENT_ID']
SERVICE_URL = os.environ['SERVICE_URL']

vertexai.init(project=PROJECT_ID, location=LOCATION)


class AdkAgent:
    SUPPORTED_CONTENT_TYPES = ['text', 'text/plain']

    def __init__(self, remote_agent):
        self._remote_agent = remote_agent

    async def stream(self, updater, query):
        session = self._remote_agent.create_session(
            user_id='default_user'
        )
        events = self._remote_agent.stream_query(
            user_id='default_user',
            session_id=session['id'],
            message=query,
        )
        text_parts = []        
        for event in events:
            if text_parts: # not a last part
                updater.new_agent_message(text_parts)
            text_parts = []
            if 'content' in event and 'parts' in event['content']:
                for part in event['content']['parts']:
                    if 'text' in part:
                        text_parts.append(TextPart(text=part['text']))
        # treat last part as an artifact
        await updater.add_artifact(text_parts)
        remote_agent.delete_session(
            user_id='default_user',
            session_id=session['id']
        )


class AdkAgentExecutor(AgentExecutor):
    def __init__(self, adk_agent: AdkAgent):
        self.adk_agent = adk_agent

    async def cancel(self, request, event_queue):
        raise ServerError(
            error=UnsupportedOperationError('Cancel operation is not supported.')
        )
        
    async def execute(self, context, event_queue):
        updater = TaskUpdater(
            event_queue,
            context.task_id,
            context.context_id
        )
        if not context.current_task:
            await updater.submit()
        await updater.start_work()

        query = context.get_user_input()
        await self.adk_agent.stream(updater, query)
        await updater.complete()


def create_agent_card(
    base_url, supported_content_types, agent_name, agent_description):
    
    capabilities = AgentCapabilities(streaming=True)
    skill = AgentSkill(
        id=f"{agent_name.lower().replace(' ', '_')}_skill",
        name=f'{agent_name} Skill',
        description=agent_description,
        tags=[tag.strip() for tag in agent_name.lower().split()],
        examples=[f'Interact with {agent_name}'],
    )

    return AgentCard(
        name=agent_name,
        description=agent_description,
        url=base_url,
        version='1.0.0',
        defaultInputModes=supported_content_types,
        defaultOutputModes=supported_content_types,
        capabilities=capabilities,
        skills=[skill],
    )


remote_agent = agent_engines.get(AGENT_ID)
adk_agent = AdkAgent(remote_agent=remote_agent)

agent_card = create_agent_card(
    base_url=SERVICE_URL,
    supported_content_types=AdkAgent.SUPPORTED_CONTENT_TYPES,
    agent_name=remote_agent.name,
    agent_description=remote_agent.name,
)
    
request_handler = DefaultRequestHandler(
    agent_executor=AdkAgentExecutor(adk_agent),
    task_store=InMemoryTaskStore(),
)

server_app_builder = A2AStarletteApplication(
    agent_card=agent_card, http_handler=request_handler
)

app = server_app_builder.build()
EOF

cat <<EOF >Dockerfile
FROM python:3.11-slim
WORKDIR /app
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONUNBUFFERED 1
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]
EOF

cat <<EOF >requirements.txt
uvicorn
starlette
google-adk==1.11.0
google-cloud-aiplatform==1.108.0
a2a-sdk==0.3.1
EOF

In [ ]:
agents = [
    'research_agent1',
    'research_agent2',
    'writer_agent',
    'review_agent',
]

deployed_agents = {
    agent.display_name: agent.name for agent in agent_engines.list()
}

for agent_name in agents:
    AGENT_ID = deployed_agents[agent_name]
    print(f'## {agent_name}: {AGENT_ID}')

    SERVICE_NAME=f'a2a-server-{agent_name}'.replace('_', '-')
    SERVICE_URL=f'https://{SERVICE_NAME}-{PROJECT_NUMBER}.{LOCATION}.run.app'

    !mkdir -p {agent_name}
    !cp main.py requirements.txt Dockerfile {agent_name}/
    !cd {agent_name} && gcloud builds submit --tag {REPO}/{SERVICE_NAME}
    !gcloud run deploy {SERVICE_NAME} \
       --image {REPO}/{SERVICE_NAME} \
       --platform managed \
       --region {LOCATION} \
       --no-allow-unauthenticated \
       --update-env-vars "\
PROJECT_ID={PROJECT_ID},REGION={LOCATION},AGENT_ID={AGENT_ID},\
SERVICE_URL={SERVICE_URL}" 2>&1 | cat

## プロキシーエージェントの作成

In [61]:
import httpx
from a2a.client import A2ACardResolver, A2AClient
from a2a.types import MessageSendParams, SendStreamingMessageRequest
import google.auth.transport.requests
import google.oauth2.id_token 


async def a2a_remote_call(query, a2a_server_url):
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, a2a_server_url)
    headers = {
        'Authorization': f'Bearer {id_token}',
        'Content-Type': 'application/json'
    }
    payload = {
        'message': {
            'role': 'user',
            'parts': [{'kind': 'text', 'text': query}],
            'messageId': uuid.uuid4().hex,
        },
    }
    request = SendStreamingMessageRequest(
        id=uuid.uuid4().hex,
        params=MessageSendParams(**payload),
    )

    async with httpx.AsyncClient(headers=headers, timeout=100) as httpx_client:
        agent_card = await A2ACardResolver(
            base_url=a2a_server_url, httpx_client=httpx_client
        ).get_agent_card()
        #card = await card_resolver.get_agent_card()
        client = A2AClient(httpx_client, agent_card, url=a2a_server_url)
        stream_response = client.send_message_streaming(request)
        result = []
        async for chunk in stream_response:
            chunk = json.loads(chunk.root.model_dump_json(exclude_none=True))
            if chunk['result']['kind'] == 'artifact-update':
                text_messages = []
                for part in chunk['result']['artifact']['parts']:
                    if 'text' in part:
                        text_messages.append(part['text'])
                result.append(Part(text='\n'.join(text_messages)))
    
    return result


def get_call_a2a_agent(agent_name):
    service_name=f'a2a-server-{agent_name}'.replace('_', '-')
    a2a_server_url=f'https://{service_name}-{PROJECT_NUMBER}.{LOCATION}.run.app'

    async def call_a2a_agent(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> Content:
        contents = [content.model_dump() for content in llm_request.contents]
        parts = await a2a_remote_call(json.dumps(contents), a2a_server_url)
        return LlmResponse(
            content=Content(role='model', parts=parts) 
        )
    return call_a2a_agent

In [62]:
research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash', # not used
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    before_model_callback=get_call_a2a_agent('research_agent1'),
)

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    before_model_callback=get_call_a2a_agent('research_agent2'),
)

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    before_model_callback=get_call_a2a_agent('writer_agent'),
)

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    before_model_callback=get_call_a2a_agent('review_agent'),
)

## 業務フローエージェントの定義

In [63]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        copy.deepcopy(research_agent1),
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        copy.deepcopy(research_agent2),
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        copy.deepcopy(writer_agent),
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        copy.deepcopy(review_agent),
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        copy.deepcopy(research_agent),
        copy.deepcopy(write_and_review_agent),
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [64]:
client = LocalApp(root_agent)
DEBUG = False

query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！私は、あなたが指定したテーマに関する記事を作成するお手伝いをします。

具体的には、まずリサーチエージェントがテーマに関する調査レポートを作成します。その後、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。

記事のテーマを指定していただければ、すぐに作成に取り掛かります。ご希望のテーマがあれば教えてください！



In [65]:
query = '''
「夏の節電対策」をテーマに記事を作成してください。
'''
result = await client.stream(query)

/var/tmp/ipykernel_21611/3628848799.py:32: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  client = A2AClient(httpx_client, agent_card, url=a2a_server_url)


かしこまりました。「夏の節電対策」をテーマにした記事の作成ですね。

まず、リサーチエージェントに調査レポートを依頼します。


---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。

「夏の節電対策」に関する記事作成のための調査レポートトピック案を以下にまとめました。

---
## 調査レポート：夏の節電対策

### 記事作成のためのトピックリスト

1.  **エアコンを賢く使う節電術**
    *   適切な設定温度と湿度の目安
    *   扇風機やサーキュレーターとの併用効果
    *   自動運転モードやタイマー機能の活用
    *   フィルター清掃や室外機の管理
2.  **その他の家電製品における節電ポイント**
    *   冷蔵庫：設置場所、開閉回数、詰め込み過ぎの防止
    *   照明：LEDへの切り替え、不要な消灯
    *   テレビ・PC：省エネモード、画面の明るさ調整、こまめな電源オフ
    *   温水洗浄便座、炊飯器、ポットなど待機電力の削減
3.  **住まいの構造と環境を活かした節電対策**
    *   窓の遮熱・断熱対策：遮光カーテン、ブラインド、すだれ、グリーンカーテンの活用
    *   日中の換気と夜間の外気利用
    *   屋根や壁の断熱・遮熱効果
4.  **電力プランの見直しとスマート技術の活用**
    *   時間帯別料金プランの活用（ピークシフト）
    *   スマートメーターやHEMS（家庭用エネルギー管理システム）による電力の見える化と制御
    *   スマート家電連携による自動節電
5.  **健康と快適さを両立させる無理のない節電方法**
    *   熱中症予防を考慮した節電の進め方
    *   クールビズや涼しい服装の工夫
    *   体を冷やすアイテム（冷感グッズ、保冷剤など）の利用
    *   入浴や食事による体温調整

---

## 選定したトピックに基づいて、調査レポートを作成します。

はい、承知いたしました。
「夏の節電対策」に関する調査レポートを作成します。

---

## 調査レポート：夏の節電対策

### 1. エアコンを賢く使う節電術

エアコンの適切な

In [66]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

記事の作成を指示します。


---
## ライターエージェントが記事を執筆します。
---



/var/tmp/ipykernel_21611/3628848799.py:32: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  client = A2AClient(httpx_client, agent_card, url=a2a_server_url)


暑い夏がやってきましたね！電気代が気になるけど、快適に過ごしたい…そんなあなたに、無理なくできる夏の節電対策をご紹介します。

## 夏の節電ってどんなこと？

夏の節電は、ただガマンするだけじゃありません。エアコンなどの家電を賢く使うのはもちろん、住まいの環境を工夫したり、最新のテクノロジーを活用したりと、様々なアプローチがあります。大切なのは、健康や快適さを犠牲にせず、効率的に電気を使うこと。室温や湿度を適切に保ちつつ、無駄な電力消費を抑えることで、家計にも地球にも優しい夏を過ごすことができますよ。

## 最新の節電トレンドを見てみよう！

最近特に注目されているのが、電力プランの見直しとスマート技術の活用です。
*   多くの電力会社が提供している「時間帯別料金プラン」を利用して、電気代が割安になる夜間に洗濯機や食洗機を使う「ピークシフト」は、賢い節電術として広まっています。
*   スマートメーターやHEMS（家庭用エネルギー管理システム）を導入すれば、リアルタイムで電力使用量を「見える化」し、自分の消費行動を把握できます。これにより、より意識的に節電に取り組んだり、HEMSが自動で家電の運転を最適化してくれたりもします。
*   スマート家電とインターネットを連携させれば、スマートフォンから外出先でエアコンの電源を切ったり、帰宅前に部屋を冷やしておいたりと、快適さと節電を両立させることが可能になっています。

## 今日からできる！実践してみよう！

今日からでもすぐに試せる実践的な節電対策として、ぜひ「エアコンと扇風機（またはサーキュレーター）の合わせ技」を試してみませんか？

環境省も推奨しているように、エアコンの設定温度は28℃を目安にしましょう。これに加えて、扇風機やサーキュレーターを使って室内の空気を循環させると、冷たい空気が部屋全体に行き渡りやすくなり、体感温度が下がります。エアコン単独で設定温度を下げ続けるよりも、ぐっと電気代を抑えながら涼しさをキープできますよ！エアコンのフィルター掃除をこまめに行うことも、効率を保つ上で非常に重要です。

## まとめ

夏の節電は、家計に優しいだけでなく、地球温暖化対策にも貢献できる持続可能なライフスタイルに繋がります。無理なく、賢く、そして快適に夏を乗り切るために、今回ご紹介した方法の中から、ぜひご自

In [67]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

/var/tmp/ipykernel_21611/3628848799.py:32: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  client = A2AClient(httpx_client, agent_card, url=a2a_server_url)


ライターエージェントとレビューエージェントに記事の修正を依頼します。


---
## ライターエージェントが記事を執筆します。
---

暑い夏がやってきましたね！電気代が気になるけれど、快適に過ごしたい…そんなあなたに、無理なくできる夏の節電対策をご紹介します。

## 夏の節電ってどんなこと？💡

夏の節電は、ただ暑さを我慢するだけではありません。エアコンなどの家電を賢く使う工夫はもちろん、住まいの環境を整えたり、最新のテクノロジーを活用したりと、様々なアプローチがあります。大切なのは、健康や快適さを犠牲にせず、効率的に電気を使うこと。室温や湿度を適切に保ちながら、無駄な電力消費を抑えることで、家計にも地球にも優しい夏を過ごすことができるんです。

## 最新の節電トレンドを見てみよう！📈

最近特に注目されているのが、電力プランの見直しとスマート技術の活用です。

*   **時間帯別料金プランの活用（ピークシフト）**
    多くの電力会社が提供している「時間帯別料金プラン」を利用して、電気代が割安になる夜間に洗濯機や食洗機を使う「ピークシフト」が賢い節電術として広まっています。
*   **「見える化」と「自動化」で意識改革！**
    スマートメーターの導入で、リアルタイムで電力使用量を確認できるようになり、自分の消費行動を「見える化」できます。さらにHEMS（家庭用エネルギー管理システム）を導入すれば、家全体の電力使用状況を一元的に管理し、家電の運転を最適化して自動で節電することも可能になります。
*   **スマホ連携で快適＆節電**
    スマート家電とインターネットを連携させれば、外出先からエアコンの電源を切ったり、帰宅前に部屋を涼しくしておいたりと、快適さと節電を両立できるようになります。消し忘れ防止にも役立ちますね。

## 今日からできる！実践してみよう！✨

さあ、今日からすぐに試せる具体的な節電術をいくつかご紹介します！

*   **エアコンと扇風機（またはサーキュレーター）の合わせ技**
    環境省も推奨しているように、エアコンの設定温度は28℃を目安にしましょう。これに加えて、扇風機やサーキュレーターを使って室内の空気を循環させると、冷たい空気が部屋全体に行き渡り、体感温度が下がります。エアコン単独で設定温度を下げ続け

In [68]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

/var/tmp/ipykernel_21611/3628848799.py:32: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  client = A2AClient(httpx_client, agent_card, url=a2a_server_url)


承知いたしました。ライターエージェントとレビューエージェントに記事の修正を依頼します。


---
## ライターエージェントが記事を執筆します。
---

# 【電気代高騰】夏を快適に乗り切る！今日からできる神節電術5選

近年、電気代の高騰が家計を圧迫し、夏の快適な生活にも影響を与えています。しかし、無理なく賢く節電する方法を知っていれば、厳しい夏も快適に乗り越えられます。この記事では、今日から実践できる効果的な節電術を具体的にご紹介します。

## 夏の節電ってどんなこと？💡

夏の節電は、ただ暑さを我慢することだけではありません。エアコンなどの家電製品を賢く使うのはもちろん、住まいの環境を工夫したり、最新のテクノロジーを活用したりと、様々なアプローチがあります。大切なのは、健康や快適さを犠牲にせず、効率的に電気を使うこと。室温や湿度を適切に保ちながら、無駄な電力消費を抑えることで、お財布にも地球にも優しい夏を過ごすことができるんです。

## 最新の節電トレンドを見てみよう！📈

最近特に注目されているのが、電力プランの見直しやスマート技術の活用です。

*   **時間帯別料金プランの活用（ピークシフト）**
    多くの電力会社が提供している「時間帯別料金プラン」を利用して、電気代が割安になる夜間に洗濯機や食洗機を使う「ピークシフト」は、賢い節電術として広まっています。
*   **「見える化」と「自動化」で意識改革！**
    スマートメーターの導入で、リアルタイムの電力使用量を確認できるようになり、自分の消費行動を「見える化」できます。さらに、HEMS（家庭用エネルギー管理システム）を導入すれば、家全体の電力使用状況を一元管理し、家電の運転を最適化して自動で節電することも可能になります。
*   **スマホ連携で快適＆節電**
    スマート家電とインターネットを連携させれば、外出先からスマートフォンのアプリでエアコンの電源を切ったり、帰宅前に部屋を冷やしておいたりといった操作が可能です。消し忘れ防止にも役立ち、快適さと節電を両立できますよ。

## 今日からできる！実践してみよう！✨

さあ、今日からすぐに試せる具体的な節電術をいくつかご紹介します！

*   **エアコンと扇風機の合わせ技で体感温度を下げよう！**
    環境省も推奨して

In [69]:
query = '''
これで完成です。完成した記事をもう一度表示して。
'''
result = await client.stream(query)

# 【電気代高騰】夏を快適に乗り切る！今日からできる神節電術5選

近年、電気代の高騰が家計を圧迫し、夏の快適な生活にも影響を与えています。しかし、無理なく賢く節電する方法を知っていれば、厳しい夏も快適に乗り越えられます。この記事では、今日から実践できる効果的な節電術を具体的にご紹介します。

## 夏の節電ってどんなこと？💡

夏の節電は、ただ暑さを我慢することだけではありません。エアコンなどの家電製品を賢く使うのはもちろん、住まいの環境を工夫したり、最新のテクノロジーを活用したりと、様々なアプローチがあります。大切なのは、健康や快適さを犠牲にせず、効率的に電気を使うこと。室温や湿度を適切に保ちながら、無駄な電力消費を抑えることで、お財布にも地球にも優しい夏を過ごすことができるんです。

## 最新の節電トレンドを見てみよう！📈

最近特に注目されているのが、電力プランの見直しやスマート技術の活用です。

*   **時間帯別料金プランの活用（ピークシフト）**
    多くの電力会社が提供している「時間帯別料金プラン」を利用して、電気代が割安になる夜間に洗濯機や食洗機を使う「ピークシフト」は、賢い節電術として広まっています。
*   **「見える化」と「自動化」で意識改革！**
    スマートメーターの導入で、リアルタイムの電力使用量を確認できるようになり、自分の消費行動を「見える化」できます。さらに、HEMS（家庭用エネルギー管理システム）を導入すれば、家全体の電力使用状況を一元管理し、家電の運転を最適化して自動で節電することも可能になります。
*   **スマホ連携で快適＆節電**
    スマート家電とインターネットを連携させれば、外出先からスマートフォンのアプリでエアコンの電源を切ったり、帰宅前に部屋を冷やしておいたりといった操作が可能です。消し忘れ防止にも役立ち、快適さと節電を両立できますよ。

## 今日からできる！実践してみよう！✨

さあ、今日からすぐに試せる具体的な節電術をいくつかご紹介します！

*   **エアコンと扇風機の合わせ技で体感温度を下げよう！**
    環境省も推奨しているように、エアコンの設定温度は28℃を目安にしましょう。これに加えて、扇風機やサーキュレーターを使って室内の空気を循環させると、冷たい空気が部屋全体に行き渡